In [1]:
%load_ext autoreload
%autoreload 2


# from backend.database.all_models import get_all_models
import pandas as pd
# get_all_models()
from backend.models import Classroom, Student, Exam, ExamResult,Module
from backend.database.database import get_db
import uuid


In [2]:

db = get_db()

def get_overall_results(classroom_id: uuid.UUID):
    results = (
        db.query(
            Classroom,
            Student,
            Module,
            Exam,
            ExamResult,
        )
        .join(Student)
        .join(ExamResult)
        .join(Exam)
        .join(Module)
        .filter(Classroom.id == classroom_id)
        .all()
    )
    
    return results


# Example usage
results = get_overall_results(uuid.UUID("fbb90108-5c1b-4a4f-9a5c-153b8f472916"))

# Convert results to a pandas DataFrame
df = pd.DataFrame(results, columns=['classroom_name', 'student_name', 'module_name', 'exam_name', 'marks_obtained', 'total_marks', 'percentage'])

# Pivot the DataFrame to get the desired format
df_pivot = df.pivot_table(index='student_name', columns='module_name', values='marks_obtained', aggfunc='sum').fillna(0)

# Calculate total marks and percentage for each student
df_pivot['TOTAL_MARKS'] = df_pivot.sum(axis=1)
df_pivot['PERCENTAGE'] = (df_pivot['TOTAL_MARKS'] / (df['total_marks'].max() * len(df_pivot.columns))) * 100
df_pivot['MEAN'] = df_pivot.mean(axis=1)

# Reset the index to make 'student_name' a column
df_pivot.reset_index(inplace=True)

# Sort by percentage in descending order
df_sorted = df_pivot.sort_values(by='PERCENTAGE', ascending=False)

print(df_sorted)

module_name     student_name Module 1 Module 2 Module 3 Module 4 Module 5  \
4            Victoria Lawson       99       41       92       59       81   
1               Holly Bryant       83       95       92       67       61   
0               Daniel Moore       89       95       59       80       50   
2              Justin Mosley       59       87       47       98       43   
3            Shelley Goodman       81       72       43       59       93   

module_name Module 6 Module 7 TOTAL_MARKS PERCENTAGE        MEAN  
4                 99      100         571   71.37500  134.819444  
1                 67       84         549   68.62500     129.625  
0                 65       53         491   61.37500  115.930556  
2                 61       89         484     60.500  114.277778  
3                 50       56         454    56.7500  107.194444  
